In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic") \
    .getOrCreate()

## Preprocessing

In [3]:
ratings = spark.read.csv("ratings_Beauty.csv", header=True, inferSchema=True)
ratings.show()

+--------------+----------+------+----------+
|        UserId| ProductId|Rating| Timestamp|
+--------------+----------+------+----------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200|
| AKJHHD5VEH7VG|0762451459|   5.0|1404518400|
|A1BG8QW55XHN6U|1304139212|   5.0|1371945600|
|A22VW0P4VZHDE3|1304139220|   5.0|1373068800|
|A3V3RE4132GKRO|130414089X|   5.0|1401840000|
|A327B0I7CYTEJC|130414643X|   4.0|1389052800|
|A1BG8QW55XHN6U|130414643X|   5.0|1372032000|
| AIFAAVTUYHEHB|130414643X|   4.0|1378252800|
| AVOGV98AYOFG2|1304146537|   5.0|1372118400|
|A22VW0P4VZHDE3|130414674X|   5.0|1371686400|
| AVOGV98AYOFG2|1304168522|   5.0|1372118400|
| A6R426V4J7AOM|1304168522|   5.0|1373414400|
|A22VW0P4VZHDE3|1304174778|   5.0|1372896000|
| AKGB62WGF35J8|1304174778|   5.0|1372896000|
|A22VW0P4VZHDE3|1304174867|   5.0|

In [4]:
ratings.createOrReplaceTempView("beauties")

In [5]:
new_id = spark.sql("SELECT DISTINCT UserId FROM beauties")
new_id.show()

+--------------+
|        UserId|
+--------------+
|A2HNQ3JHXDSVMW|
|A2DOQ89OLXNHNL|
|A17U6P3YQISHYH|
|A189H8NPEQN58F|
|A3UMVASITREVNQ|
|A20DHG878IX2OE|
|A1GAMBKLVMBTAC|
|A3DDSAGWD0UN9H|
| AY8MX1BFYICXC|
|A2LF9DJ24VWY6O|
|A2QPKW46MNWEBY|
|A389H99XZRUOWE|
|A222CH7IMYYY49|
| AKDBMU0AIIBZV|
|A2G6WDE7VCO8WR|
|A263E8JYWMRP5V|
|A2823NUWUHXMJM|
|A18FTRFQQ141CP|
|A2651ZXBNPFTSZ|
|A3MG2ME0SETH33|
+--------------+
only showing top 20 rows



In [6]:
new_id.createOrReplaceTempView("newId")

In [7]:
new_user_id = spark.sql("SELECT UserId, ROW_NUMBER() OVER (ORDER BY UserId) AS user_id FROM newId")
new_user_id.show()

+--------------------+-------+
|              UserId|user_id|
+--------------------+-------+
|A00008821J0F472ND...|      1|
|A000186437REL8X2R...|      2|
|A0002574WYJMBWKNCPY8|      3|
|A00029263J863WSR0...|      4|
|A00031961JI1CBNV98TW|      5|
|A000325234LCBTFVL...|      6|
|A0003526DBE3EVU1DXE5|      7|
|A0004690232S2SX49...|      8|
|A000622834IP6B056...|      9|
|A0007094NAUEOIOA53YH|     10|
|A00088021N0VU5Z2U...|     11|
|A0010876CNE3ILIM9HV0|     12|
|A0011102257KBXODK...|     13|
|A001114613O3F18Q5...|     14|
|A00116502XCN8ZW09...|     15|
|A00120381FL204MYH...|     16|
|A001235832OWO8HZG...|     17|
|A00125322X21CGQBJ...|     18|
|A00126503SUWI86KZ...|     19|
|A00144702V3Q8N2EJ...|     20|
+--------------------+-------+
only showing top 20 rows



In [8]:
ratings = ratings.join(new_user_id, ratings.UserId == new_user_id.UserId)
ratings.show()

+--------------------+----------+------+----------+--------------------+-------+
|              UserId| ProductId|Rating| Timestamp|              UserId|user_id|
+--------------------+----------+------+----------+--------------------+-------+
|A0061198154UGJDK6...|B007UK1ABG|   5.0|1378252800|A0061198154UGJDK6...|     98|
|A0061198154UGJDK6...|B009Q1YPV4|   1.0|1378252800|A0061198154UGJDK6...|     98|
|A022858327OPV8WND...|B00AG4QCAM|   1.0|1402358400|A022858327OPV8WND...|    361|
|A0232781C6JWW3M9F7LP|B0036B3C0A|   1.0|1385337600|A0232781C6JWW3M9F7LP|    368|
|A0232781C6JWW3M9F7LP|B003JN6I0G|   1.0|1400803200|A0232781C6JWW3M9F7LP|    368|
|A0232781C6JWW3M9F7LP|B009CJHXDW|   1.0|1391990400|A0232781C6JWW3M9F7LP|    368|
|A0232781C6JWW3M9F7LP|B00DZZETZ4|   1.0|1385337600|A0232781C6JWW3M9F7LP|    368|
|A02680541GF3IVW82...|B002BGDLDO|   5.0|1393977600|A02680541GF3IVW82...|    418|
|A02680541GF3IVW82...|B004ZC13OG|   4.0|1393977600|A02680541GF3IVW82...|    418|
|A02680541GF3IVW82...|B00538

In [9]:
ratings = ratings.select('user_id', 'ProductId', 'Rating')

In [10]:
ratings = ratings.withColumnRenamed('user_id', 'UserId')
ratings.show()

+------+----------+------+
|UserId| ProductId|Rating|
+------+----------+------+
|    98|B007UK1ABG|   5.0|
|    98|B009Q1YPV4|   1.0|
|   361|B00AG4QCAM|   1.0|
|   368|B0036B3C0A|   1.0|
|   368|B003JN6I0G|   1.0|
|   368|B009CJHXDW|   1.0|
|   368|B00DZZETZ4|   1.0|
|   418|B002BGDLDO|   5.0|
|   418|B004ZC13OG|   4.0|
|   418|B00538TSMU|   5.0|
|   418|B0073GFTBO|   5.0|
|   418|B00BQ76XK2|   3.0|
|   894|B0092LDWWQ|   4.0|
|  1053|B0065JCV84|   5.0|
|  1134|B007XBR8HW|   1.0|
|  1275|B00L5JHZJO|   5.0|
|  1769|B001AO0WCG|   5.0|
|  2035|B001LQY1X4|   3.0|
|  2098|B002RL8IZ4|   5.0|
|  2171|B000GCW3XE|   5.0|
+------+----------+------+
only showing top 20 rows



In [15]:
query1 = ratings.select('UserId', 'ProductId', 'Rating')

In [16]:
query1.show()

+------+----------+------+
|UserId| ProductId|Rating|
+------+----------+------+
|    98|B007UK1ABG|   5.0|
|    98|B009Q1YPV4|   1.0|
|   361|B00AG4QCAM|   1.0|
|   368|B0036B3C0A|   1.0|
|   368|B003JN6I0G|   1.0|
|   368|B009CJHXDW|   1.0|
|   368|B00DZZETZ4|   1.0|
|   418|B002BGDLDO|   5.0|
|   418|B004ZC13OG|   4.0|
|   418|B00538TSMU|   5.0|
|   418|B0073GFTBO|   5.0|
|   418|B00BQ76XK2|   3.0|
|   894|B0092LDWWQ|   4.0|
|  1053|B0065JCV84|   5.0|
|  1134|B007XBR8HW|   1.0|
|  1275|B00L5JHZJO|   5.0|
|  1769|B001AO0WCG|   5.0|
|  2035|B001LQY1X4|   3.0|
|  2098|B002RL8IZ4|   5.0|
|  2171|B000GCW3XE|   5.0|
+------+----------+------+
only showing top 20 rows



In [19]:
# Convert to Pandas
import pandas as pd
query1Pandas = query1.toPandas()

In [22]:
# Save to single CSV
query1Pandas.to_csv("D:/Tools/FP/dataset/beauty_ratings.csv", index=False)